In [1]:



import pandas as pd
import numpy as np
import jsonlines
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch_optimizer as optim


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from importlib import reload
pd.set_option('display.max_rows', 500)
pd.set_option('display.float_format', '{:0.3f}'.format)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.width = 0
import warnings
import torchvision
warnings.filterwarnings('ignore')

from facebook_hateful_memes_detector.utils.globals import set_global, get_global
set_global("cache_dir", "/home/ahemf/cache/cache")

from facebook_hateful_memes_detector.utils import read_json_lines_into_df, in_notebook, set_device
get_global("cache_dir")
from facebook_hateful_memes_detector.models import Fasttext1DCNNModel, MultiImageMultiTextAttentionEarlyFusionModel, LangFeaturesModel, AlbertClassifer
from facebook_hateful_memes_detector.preprocessing import TextImageDataset, my_collate, get_datasets, get_image2torchvision_transforms, TextAugment
from facebook_hateful_memes_detector.preprocessing import DefinedRotation, QuadrantCut, ImageAugment
from facebook_hateful_memes_detector.training import *
import facebook_hateful_memes_detector
reload(facebook_hateful_memes_detector)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
set_device(device)


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
set_device(device)
device


device(type='cuda')

In [3]:
aug_speeds = {"keyboard": 117, "char_substitute": 109, "char_insert": 109, "char_swap": 114,
              "ocr": 114, "char_delete": 108,
              "word_insert": 0.0, "word_substitute": 0.0, "text_rotate": 32,
              "stopword_insert": 34, "word_join": 32, "word_cutout": 36,
              "w2v_insert": 0.0, "w2v_substitute": 0.0, 
              "fasttext": 137, "glove_twitter": 88, "glove_wiki": 82, "word2vec": 137,
              "synonym": 522, "split": 110, "sentence_shuffle": 67, "one_third_cut": 0.0, "half_cut":0.0}


In [4]:


choice_probas = {"keyboard": 0.1, "char_substitute": 0.0, "char_insert": 0.1, "char_swap": 0.1, "ocr": 0.0, "char_delete": 0.1,
                 "fasttext": 0.0, "glove_twitter": 0.0, "glove_wiki": 0.0, "word2vec": 0.0, "split": 0.1,
                 "stopword_insert": 0.3, "word_join": 0.1, "word_cutout": 0.8,
                 "text_rotate": 0.5, "sentence_shuffle": 0.5, "one_third_cut": 0.3, "half_cut":0.1}
preprocess_text = TextAugment([0.05, 0.05, 0.05, 0.35, 0.3, 0.2], choice_probas, fasttext_file="wiki-news-300d-1M-subword.bin")

im_transform = transforms.RandomChoice([transforms.Grayscale(num_output_channels=3), 
                                        transforms.RandomHorizontalFlip(p=1.0),
                                        lambda x: x,
                                        transforms.Compose([transforms.Resize(256), transforms.CenterCrop(224)]),
                                        transforms.Compose([transforms.Resize(480), transforms.CenterCrop(400)]),
                                        transforms.Compose([transforms.Resize(640), transforms.CenterCrop(520)]),
                                        DefinedRotation(15), QuadrantCut()])

data = get_datasets(data_dir="../data/", train_text_transform=preprocess_text, train_image_transform=im_transform, 
                    test_text_transform=None, test_image_transform=None, 
                    cache_images = True, use_images = True, dev=False, 
                    keep_original_text=False, keep_original_image=False, 
                    keep_processed_image=True, keep_torchvision_image=False,)

# ImageAugment([0.2, 0.5, 0.3])


https://discuss.pytorch.org/t/how-to-implement-torch-optim-lr-scheduler-cosineannealinglr/28797/11

# Params

In [5]:
sgd = torch.optim.SGD
sgd_params = dict(lr=2e-2, momentum=0.9, dampening=0, weight_decay=0, nesterov=False)

rangerQH = optim.RangerQH
rangerQHparams = dict(lr=1e-3, betas=(0.9, 0.999), nus=(.7, 1.0),
    weight_decay=0.0,
    k=6,
    alpha=.5,
    decouple_weight_decay=True,
    eps=1e-8,)

adam = torch.optim.Adam
adam_params = params=dict(lr=1e-3, weight_decay=1e-6)

adamw = torch.optim.AdamW
adamw_params = dict(lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-2)

novograd = optim.NovoGrad
novograd_params = dict(lr= 1e-3,
    betas=(0.9, 0.999),
    eps=1e-8,
    weight_decay=0,
    grad_averaging=False,
    amsgrad=False,)

qhadam = optim.QHAdam
qhadam_params = dict(lr= 1e-3,
    betas=(0.9, 0.999),
    nus=(1.0, 1.0),
    weight_decay=0,
    decouple_weight_decay=False,
    eps=1e-8,)

radam = optim.RAdam
radam_params = dict(lr= 1e-3,
    betas=(0.9, 0.999),
    eps=1e-8,
    weight_decay=0,)

yogi = optim.Yogi
yogi_params = dict(lr= 1e-2,
    betas=(0.9, 0.999),
    eps=1e-3,
    initial_accumulator=1e-6,
    weight_decay=0)




In [6]:
batch_size=512
epochs = 10
optimizer = adam
optimizer_params = adam_params

scheduler_init_fn = get_multistep_lr([11, 13], gamma=0.25) # get_cosine_schedule_with_warmup # get_cosine_with_hard_restarts_schedule_with_warmup
scheduler_init_fn = get_cosine_schedule_with_warmup()
augmentation_weights = {"None": 1.0}



# Fasttext 1D CNN

## 512 DIMS

In [ ]:
model_fn = model_builder(Fasttext1DCNNModel, 
                         dict(classifier_dims=256, num_classes=2, 
                              n_layers=2, final_layer_builder=fb_1d_loss_builder,
                              gaussian_noise=0.2, dropout=0.1, embedding_dims=256, internal_dims=512,
                             fasttext_file="crawl-300d-2M-subword.bin", featurizer="transformer",),
                         optimiser_class=optimizer, optimiser_params=optimizer_params)





n_tests = 1
multi_eval=False 
kfold=False
results, prfs = train_validate_ntimes(model_fn, data, batch_size, epochs, 
                                      augmentation_weights=augmentation_weights, 
                                      kfold=kfold, multi_eval=multi_eval,
                                      scheduler_init_fn=scheduler_init_fn, 
                                      validation_epochs=[1, 4, 7, 9, 11, 14, 17, 19])
r1, p1 = results, prfs
results
prfs
 


# Lang Features Model

In [7]:
epochs = 5
batch_size=256

all_caps = ["fasttext_crawl", "spacy", "full_view", "key_phrases", "nltk", "gensim"] # "snlp", "ibm_max", "tmoji", "key_phrases", "full_view", "spacy", "nltk", "fasttext_crawl"
# all_caps = ["fasttext_crawl", ]
model_fn = model_builder(LangFeaturesModel, 
                         dict(classifier_dims=256, num_classes=2, 
                              gaussian_noise=0.2, dropout=0.1, embedding_dims=256, internal_dims=512,
                              capabilities=all_caps, featurizer="transformer", n_layers=2, 
                              final_layer_builder=fb_1d_loss_builder),
                         optimiser_class=optimizer, optimiser_params=optimizer_params)



n_tests = 1
multi_eval=False
kfold=False
results, prfs = train_validate_ntimes(model_fn, data, batch_size, epochs, 
                                      augmentation_weights=augmentation_weights, 
                                      kfold=kfold, multi_eval=multi_eval,
                                      scheduler_init_fn=scheduler_init_fn, 
                                      validation_epochs=[1, 4, 7, 9, 11, 14, 17, 19])
r1, p1 = results, prfs
results
prfs


Model Params = 15501232 
 LangFeaturesModel(
  (spacy_nn): ExpandContract(
    (nn): Sequential(
      (0): Transpose()
      (1): Dropout(p=0.1, inplace=False)
      (2): Conv1d(282, 256, kernel_size=(1,), stride=(1,), groups=2, bias=False)
      (3): LeakyReLU(negative_slope=0.01)
      (4): Dropout(p=0.1, inplace=False)
      (5): Conv1d(256, 128, kernel_size=(1,), stride=(1,), groups=4, bias=False)
      (6): Transpose()
      (7): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    )
  )
  (crawl_nn): ExpandContract(
    (nn): Sequential(
      (0): Transpose()
      (1): Dropout(p=0.1, inplace=False)
      (2): Conv1d(600, 384, kernel_size=(1,), stride=(1,), groups=4, bias=False)
      (3): LeakyReLU(negative_slope=0.01)
      (4): Dropout(p=0.1, inplace=False)
      (5): Conv1d(384, 192, kernel_size=(1,), stride=(1,), groups=4, bias=False)
      (6): Transpose()
      (7): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
    )
  )
  (gensim_nn): ExpandContract(
    (n


Epoch =  1 Loss = 1.294604 LR = 0.00067442
Epoch =  1 Train = 0.584293 Val = 0.585680



Epoch =  2 Loss = 0.771310 LR = 0.00094758



Epoch =  3 Loss = 0.766091 LR = 0.00060697



Epoch =  4 Loss = 0.733809 LR = 0.00018654
Epoch =  4 Train = 0.656464 Val = 0.646241



Epoch =  5 Loss = 0.708022 LR = 0.00000000



train   val
mean_or_std metric               
mean        map       0.575 0.546
            accuracy  0.469 0.463
            auc       0.690 0.672
std         map       0.000 0.000
            accuracy  0.000 0.000
            auc       0.000 0.000

train                             val                      
    precision recall    f1  supoort precision recall    f1  supoort
neg     0.801  0.214 0.338 4560.000     0.811  0.199 0.320 1140.000
pos     0.401  0.908 0.556 2640.000     0.399  0.920 0.557  660.000

# BERT Models

In [9]:
epochs = 5
batch_size=256

model_fn = model_builder(AlbertClassifer, 
                         dict(classifier_dims=256, num_classes=2, embedding_dims=768,
                              gaussian_noise=0.6, dropout=0.0, internal_dims=512,
                              final_layer_builder=fb_1d_loss_builder, n_layers=2, 
                              featurizer="transformer", model='albert-base-v2'),
                         optimiser_class=optimizer, optimiser_params=optimizer_params)



n_tests = 1
multi_eval=False
kfold=False
results, prfs = train_validate_ntimes(model_fn, data, batch_size, epochs, 
                                      augmentation_weights=augmentation_weights, 
                                      kfold=kfold, multi_eval=multi_eval,
                                      scheduler_init_fn=scheduler_init_fn, 
                                      validation_epochs=[1, 4, 7, 9, 11, 14, 17, 19])
r1, p1 = results, prfs
results
prfs



Model Params = 15256576 
 AlbertClassifer(
  (model): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_fea


Epoch =  1 Loss = 1.011382 LR = 0.00067442
Epoch =  1 Train = 0.588069 Val = 0.602994



Epoch =  2 Loss = 0.780443 LR = 0.00094758



Epoch =  3 Loss = 0.704513 LR = 0.00060697



Epoch =  4 Loss = 0.682217 LR = 0.00018654
Epoch =  4 Train = 0.649416 Val = 0.660894



Epoch =  5 Loss = 0.660487 LR = 0.00000000



train   val
mean_or_std metric               
mean        map       0.540 0.574
            accuracy  0.625 0.623
            auc       0.661 0.672
std         map       0.000 0.000
            accuracy  0.000 0.000
            auc       0.000 0.000

train                             val                      
    precision recall    f1  supoort precision recall    f1  supoort
neg     0.728  0.652 0.688 4560.000     0.731  0.640 0.683 1140.000
pos     0.491  0.580 0.532 2640.000     0.489  0.594 0.536  660.000

# Predict

In [ ]:
model_fn = model_builder(Fasttext1DCNNModel, 
                         dict(classifer_dims=256, num_classes=2, 
                              gaussian_noise=0.3, dropout=0.2, embedding_dims=512, internal_dims=256,
                             fasttext_file="crawl-300d-2M-subword.bin", classifier="transformer",),
                         optimiser_class=optimizer, optimiser_params=optimizer_params)


In [ ]:
batch_size=512
epochs = 1

submission, text_model = train_and_predict(model_fn, data, batch_size, epochs, augmentation_weights, scheduler_init_fn=scheduler_init_fn)
submission.to_csv("submission.csv",index=False)
submission.sample(3)


In [ ]:
submission.sample(10)